# #. Group Units

A Group is a collection of Group Units. A Group Unit consists of:
- **Nonce**: A unique identifier for the Group Unit.  Also, declares the Group Unit's Hierarchical Deterministic (HD) path.
- **Owner**: The owner of the Group Unit.  Generally, the owner is the Group's creator, but alternative ownership models are possible.
- **Credential**: A credential that is used to authenticate the Group Unit and allow access to the Group Unit's data.
- **Data**: The data that is stored in the Group Unit.

In [4]:
import sys

sys.path.append("../../forme-groups-python-3-12/")

## Controller

The Controller is the entity that creates the Group.  The Controller is responsible for:
- Creating the Group.
- Creating the Group Units.

In [5]:
from src.groups.controller import Controller
from src.groups.unit import Data

# Create a Controller with a fresh group unit pool
controller = Controller()

# Create some data to store in the Group
data = Data((1, 2, 3))

# Create a GroupUnit and store the data in it
group_unit = controller._create_group_unit(data)

print(group_unit)

is_sub_unit: None, super_unit_schema: None, super_unit_hash: None
next_nonce: 1
active: nonce: 0, owner: , credential: , data: entry: (0,), schema: None
base_values: (BaseValue(value=0, type=int),)
predicted_super_nonce: 0
predicted_super_nonce_group_unit: nonce: 0, owner: , credential: , data: entry: (0,), schema: None
predicted_super_nonce_group_unit_schema: None
schema_to_enforce: None
data: entry: (1, 2, 3), schema: None
group_unit_hash: d63fb21887e17825aecdb8aff280d9a8f238ba808b461f0455521b31d95275e2, nonce_hash: 3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a37fd
hash_item: d63fb21887e17825aecdb8aff280d9a8f238ba808b461f0455521b31d95275e2
hash_item: 3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a37fd
nonce: 1, owner: , credential: , data: entry: (1, 2, 3), schema: None


## Create Sub Units from a schema



In [6]:
import uuid
import random
from src.groups.base import SchemaEntry, BaseSchema

# Create a SchemaEntry for the GroupUnit
schema_entry = SchemaEntry("name", "str")
schema_entry2 = SchemaEntry("age", "int")
schema_for_sub_units = BaseSchema((schema_entry, schema_entry2))

# Create the data to store in the GroupUnit
data = Data(('a', 'b', 'c'), schema_for_sub_units)

# Create a GroupUnit and store the data in it
group_unit = controller._create_group_unit(data)
print(group_unit)

# Create the data with the proper schema
sub_unit_data = Data(("Fred", 12))

sub_group_unit = controller._create_group_unit(sub_unit_data, True)

print(sub_group_unit)

sub_unit_data2 = Data(("Jane", 11))

sub_group_unit2 = controller._create_group_unit(sub_unit_data2)

print(sub_group_unit2)

sub_unit3 = Data(("Jill", 10))
print(sub_unit3)
sub_group_unit3 = controller._create_group_unit(sub_unit3)

for i in range(0, 100):
    random_data = Data((str(uuid.uuid4()), random.randint(0, 1000)))
    print(controller.pool.group_units)
    print(random_data)
    random_group_unit = controller._create_group_unit(random_data)
    print(random_group_unit)


is_sub_unit: None, super_unit_schema: None, super_unit_hash: None
next_nonce: 2
active: nonce: 1, owner: , credential: , data: entry: (1, 2, 3), schema: None
base_values: (BaseValue(value=1, type=int),)
predicted_super_nonce: 1
predicted_super_nonce_group_unit: nonce: 1, owner: , credential: , data: entry: (1, 2, 3), schema: None
predicted_super_nonce_group_unit_schema: None
schema_to_enforce: None
data: entry: ('a', 'b', 'c'), schema: (key=name, value=str), (key=age, value=int)
group_unit_hash: d3372f6d5dd15f662c751fa8ff9ceec8f3fd8699478dc02f3391f7016e216992, nonce_hash: 32ee78186a3407f4f288673b1a7dca6154c294f435f444ee3ba054356a88a1e8
hash_item: d3372f6d5dd15f662c751fa8ff9ceec8f3fd8699478dc02f3391f7016e216992
hash_item: 32ee78186a3407f4f288673b1a7dca6154c294f435f444ee3ba054356a88a1e8
nonce: 2, owner: , credential: , data: entry: ('a', 'b', 'c'), schema: (key=name, value=str), (key=age, value=int)
is_sub_unit: True, super_unit_schema: None, super_unit_hash: None
next_nonce: 2.0
active: